In [1]:
import tensorflow as tf
from PIL import Image
import numpy as np
import time
from PIL import Image, ImageDraw

In [2]:
path = "./posenet_mobilenet_v1_075_353_481_quant_decoder.tflite"
interpreter = tf.lite.Interpreter(path)
input_details = interpreter.get_input_details()
print(str(input_details))
output_details = interpreter.get_output_details()
print(str(output_details))

[{'name': 'sub_2', 'index': 40, 'shape': array([  1, 353, 481,   3], dtype=int32), 'shape_signature': array([  1, 353, 481,   3], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.0078125, 128), 'quantization_parameters': {'scales': array([0.0078125], dtype=float32), 'zero_points': array([128], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'poses', 'index': 36, 'shape': array([ 1, 10, 17,  2], dtype=int32), 'shape_signature': array([ 1, 10, 17,  2], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'poses:1', 'index': 37, 'shape': array([ 1, 10, 17], dtype=int32), 'shape_signature': array([ 1, 10, 17], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points'

In [10]:
# check outputstrids:  resolution = ((InputImageSize - 1) / OutputStride) + 1
output_stridex = (input_details[0]['shape'][1]-1)/(output_details[0]['shape'][1]-1)
output_stridey = (input_details[0]['shape'][2]-1)/(output_details[0]['shape'][2]-1)
print("outputstrides:", output_stridex, output_stridey)

outputstrides: 32.0 32.0


In [11]:
output_stride = 32
num_keypoints = 17

In [4]:
interpreter.allocate_tensors()

RuntimeError: Encountered unresolved custom op: PosenetDecoderOp.Node number 32 (PosenetDecoderOp) failed to prepare.


In [14]:
t1 = "./test.png"
timg = Image.open(t1)
timg2 = timg.resize((257,257))
tim = np.array(timg2)
tim2 = np.expand_dims(tim, axis=0)

In [15]:
t1 = time.time()
interpreter.set_tensor(input_details[0]['index'], tim2.astype(np.float32))
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
t2 = time.time()
print("time:",t2-t1)

time: 0.1220707893371582


In [16]:
heatmaps = np.squeeze(interpreter.get_tensor(output_details[0]['index']))
offsets = np.squeeze(interpreter.get_tensor(output_details[1]['index']))
dis1= np.squeeze(interpreter.get_tensor(output_details[2]['index']))
dis2= np.squeeze(interpreter.get_tensor(output_details[3]['index']))

In [17]:
heatmaps.shape, offsets.shape,dis1.shape, dis2.shape

((9, 9, 17), (9, 9, 34), (9, 9, 32), (9, 9, 32))

In [25]:
conf1, conf2, coords = decode_multiple_poses(heatmaps, 
                offsets, dis1, dis2, output_stride, max_pose_detections=1)

In [28]:
coords

array([[[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]]])

In [26]:
conf1

array([0.])

In [ ]:
def visual(imgpath, kps):
    timg = Image.open(imgpath)
    w,h = timg.size
    draw = ImageDraw.Draw(timg)
    for row, col in kps:
        row = int(row/17.0*w)
        col = int(col/23.0*h)
        draw.arc((col-2,row-2, col+2, row+2), 0,360, fill='red', width=1)
    return timg

In [ ]:
# refer this code:  https://github.com/rwightman/posenet-python/blob/master/posenet/decode_multi.py

In [ ]:
# blog: https://towardsdatascience.com/optimizing-pose-estimation-on-the-coral-edge-tpu-d331c63cfed

In [ ]:
# another proach is to use js in android